In [1]:
# import required packages
# from fastai2.tabular.all import *
import xgboost as xgb
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from helpers import *

pd.options.display.max_columns = None

In [2]:
# path to project directory
path = Path('./')

In [3]:
# read in training dataset
train_df = pd.read_csv(path/'data/train_v7.csv', index_col=0, dtype={'season':str,
                                                                     'squad':str,
                                                                     'comp':str})

In [9]:
train_df[train_df['player'] == 'Ben Davies'].tail(10)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,assists,bonus,bps,clean_sheets,creativity,goals_conceded,goals_scored,ict_index,influence,own_goals,penalties_missed,penalties_saved,red_cards,saves,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,relative_market_value_team_season,relative_market_value_opponent_team_season,date,squad,comp,shots_total,shots_on_target,touches,pressures,tackles,interceptions,blocks,xg,npxg,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles
101543,Ben Davies,20,2,90,Tottenham Hotspur,Liverpool,1.651906,2.487440,True,1,0,0,12,0,2.5,3,0,1.6,13.4,0,0,0,0,0,65031,3,1,0.0,-4056,1469,5525,0,2021-01-28T20:00:00Z,2021,1.0,NaN,NaN,2021-01-28,Tottenham,Premier League,0.0,0.0,72.0,15.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,53.0,60.0,88.3,39.0,0.0,0.0
102188,Ben Davies,21,2,90,Tottenham Hotspur,Brighton and Hove Albion,1.662924,0.550097,False,2,0,0,16,0,13.3,1,0,3.0,17.0,0,0,0,0,0,64460,0,1,0.0,-816,2258,3074,0,2021-01-31T19:15:00Z,2021,1.0,NaN,NaN,2021-01-31,Tottenham,Premier League,0.0,0.0,67.0,15.0,3.0,1.0,1.0,0.0,0.0,0.1,1.0,0.0,43.0,54.0,79.6,42.0,3.0,3.0
102838,Ben Davies,22,2,90,Tottenham Hotspur,Chelsea,1.628650,1.818752,True,2,0,0,10,0,1.9,1,0,1.3,11.2,0,0,0,0,0,63698,1,0,0.0,-966,1026,1992,0,2021-02-04T20:00:00Z,2021,1.0,NaN,NaN,2021-02-04,Tottenham,Premier League,0.0,0.0,73.0,12.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,41.0,63.0,65.1,41.0,0.0,0.0
102839,Ben Davies,22,2,0,Liverpool,Brighton and Hove Albion,2.534688,0.563176,True,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0,0,1280,1,0,0.0,1219,1280,61,0,2021-02-03T20:15:00Z,2021,1.0,NaN,NaN,2021-02-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103501,Ben Davies,23,2,90,Tottenham Hotspur,West Bromwich Albion,1.628650,0.299523,True,6,0,0,23,1,3.8,0,0,1.7,9.6,0,0,0,0,0,66770,0,2,4.0,2504,4436,1932,0,2021-02-07T12:00:00Z,2021,1.0,NaN,NaN,2021-02-07,Tottenham,Premier League,0.0,0.0,58.0,8.0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,39.0,47.0,83.0,33.0,0.0,1.0
103502,Ben Davies,23,2,0,Liverpool,Manchester City,2.534688,2.398901,True,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0,0,2496,4,1,0.0,1004,1472,468,0,2021-02-07T16:30:00Z,2021,1.0,NaN,NaN,2021-02-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104189,Ben Davies,24,2,90,Tottenham Hotspur,Manchester City,1.627545,2.397273,False,0,0,0,10,0,2.7,3,0,1.2,8.2,0,0,0,0,0,65448,0,3,1.0,-1433,2738,4171,1,2021-02-13T17:30:00Z,2021,1.0,NaN,NaN,2021-02-13,Tottenham,Premier League,1.0,0.0,48.0,13.0,2.0,2.0,2.0,0.0,0.0,0.0,1.0,0.0,30.0,36.0,83.3,33.0,0.0,0.0
104190,Ben Davies,24,2,0,Liverpool,Leicester City,2.532968,1.059550,False,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0,0,2393,1,3,0.0,-255,442,697,0,2021-02-13T12:30:00Z,2021,1.0,NaN,NaN,2021-02-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104976,Ben Davies,25,2,0,Tottenham Hotspur,West Ham United,1.627545,0.613679,False,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0,0,70326,1,2,0.0,4238,8138,3900,0,2021-02-21T12:00:00Z,2021,1.0,NaN,NaN,2021-02-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104977,Ben Davies,25,2,0,Liverpool,Everton,2.532968,1.111566,True,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0,0,1929,2,0,0.0,-532,110,642,0,2021-02-20T17:30:00Z,2021,1.0,NaN,NaN,2021-02-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# train_df.iloc[103502, 38:] = np.nan

In [10]:
# train_df.to_csv(path/'train_v7.csv')

In [11]:
train_df.shape

(105642, 58)

In [12]:
# features for model
season = '2021'
cat_vars = ['season', 'position', 'was_home']
cont_vars = ['gw', 'minutes']
dep_var = ['total_points']

# find the latest gameweek
last_gw = train_df['gw'][train_df['season'] == season].max()

if np.isnan(last_gw): 
    next_gw = 1 
else: 
    next_gw = last_gw + 1
    
next_gw

26

In [13]:
# read in remaining_season.csv and sort
remaining_season_df = pd.read_csv(path/'data/remaining_season.csv', 
                                  index_col=0).sort_values(by=['gw', 'team'])

# update fields needed for processing
remaining_season_df['season'] = season
remaining_season_df['total_points'] = 0

# need to add an ordered date-like field  
remaining_season_df['kickoff_time'] = [str(9000 + x) + '-99-99' for x in remaining_season_df['match_no']]

# remove match_no, don't need it anymore
remaining_season_df = remaining_season_df.drop('match_no', axis=1)

# # append remaining season to end of training set
train_df = train_df.append(remaining_season_df, sort=False).reset_index(drop=True)

In [14]:
# # read in remaining_season.csv and sort
# remaining_season_df = pd.read_csv(path/'data/remaining_season.csv', 
#                                   index_col=0).sort_values(by=['gw', 'team'])

# remaining_season_df['season'] = season
# remaining_season_df['total_points'] = 0
# remaining_season_df['kickoff_time'] = '9999-99-99'

# # append remaining season to end of training set
# train_df = train_df.append(remaining_season_df, sort=False).reset_index(drop=True)

In [15]:
# add player lag features
lag_train_df, team_lag_vars = team_lag_features(train_df, ['total_points'], ['all', 3, 5, 10, 20])
lag_train_df, player_lag_vars = player_lag_features(lag_train_df, ['total_points'], ['all', 1, 2, 3, 5, 10, 20])

# add team conceded fields for opposition
pattern = re.compile('total_points_team_pg_last_.*_opponent')
team_lag_vars += [x.replace('team', 'team_conceded') for x in team_lag_vars if pattern.match(x)]

# but remove scored for opposition
pattern = re.compile('total_points_team_pg_last_.*_opponent')
team_lag_vars = [x for x in team_lag_vars if not pattern.match(x)]

In [16]:
team_lag_vars

['total_points_team_pg_last_all',
 'total_points_team_pg_last_3',
 'total_points_team_pg_last_5',
 'total_points_team_pg_last_10',
 'total_points_team_pg_last_20',
 'total_points_team_conceded_pg_last_all_opponent',
 'total_points_team_conceded_pg_last_3_opponent',
 'total_points_team_conceded_pg_last_5_opponent',
 'total_points_team_conceded_pg_last_10_opponent',
 'total_points_team_conceded_pg_last_20_opponent']

In [17]:
# we want to set gw and season as ordered categorical variables
# need lists with ordered categories
ordered_gws = list(range(1,39))
ordered_seasons = ['1617', '1718', '1819', '1920', '2021']

# set as categories with correct order 
lag_train_df['gw'] = lag_train_df['gw'].astype('category')
lag_train_df['season'] = lag_train_df['season'].astype('category')

lag_train_df['gw'].cat.set_categories(ordered_gws, ordered=True, inplace=True)
lag_train_df['season'].cat.set_categories(ordered_seasons, ordered=True, inplace=True)

In [18]:
lag_train_df[lag_train_df['player'] == 'Bernd Leno'].tail(25)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,assists,bonus,bps,clean_sheets,creativity,goals_conceded,goals_scored,ict_index,influence,own_goals,penalties_missed,penalties_saved,red_cards,saves,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,relative_market_value_team_season,relative_market_value_opponent_team_season,date,squad,comp,shots_total,shots_on_target,touches,pressures,tackles,interceptions,blocks,xg,npxg,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,price,short_name,total_points_team,total_points_team_conceded,total_points_team_last_all,total_points_team_conceded_last_all,total_points_team_pg_last_all,total_points_team_conceded_pg_last_all,total_points_team_last_3,total_points_team_conceded_last_3,total_points_team_pg_last_3,total_points_team_conceded_pg_last_3,total_points_team_last_5,total_points_team_conceded_last_5,total_points_team_pg_last_5,total_points_team_conceded_pg_last_5,total_points_team_last_10,total_points_team_conceded_last_10,total_points_team_pg_last_10,total_points_team_conceded_pg_last_10,total_points_team_last_20,total_points_team_conceded_last_20,total_points_team_pg_last_20,total_points_team_conceded_pg_last_20,total_points_team_opponent,total_points_team_conceded_opponent,total_points_team_last_all_opponent,total_points_team_conceded_last_all_opponent,total_points_team_pg_last_all_opponent,total_points_team_conceded_pg_last_all_opponent,total_points_team_last_3_opponent,total_points_team_conceded_last_3_opponent,total_points_team_pg_last_3_opponent,total_points_team_conceded_pg_last_3_opponent,total_points_team_last_5_opponent,total_points_team_conceded_last_5_opponent,total_points_team_pg_last_5_opponent,total_points_team_conceded_pg_last_5_opponent,total_points_team_last_10_opponent,total_points_team_conceded_last_10_opponent,total_points_team_pg_last_10_opponent,total_points_team_conceded_pg_last_10_opponent,total_points_team_last_20_opponent,total_points_team_conceded_last_20_opponent,total_points_team_pg_last_20_opponent,total_points_team_conceded_pg_last_20_opponent,minutes_last_all,minutes_last_1,minutes_last_2,minutes_last_3,minutes_last_5,minutes_last_10,minutes_last_20,total_points_last_all,total_points_pg_last_all,total_points_last_1,total_points_pg_last_1,total_points_last_2,total_points_pg_last_2,total_points_last_3,total_points_pg_last_3,total_points_last_5,total_points_pg_last_5,total_points_last_10,total_points_pg_last_10,total_points_last_20,total_points_pg_last_20
97918,Bernd Leno,14,1,90.00,Arsenal,Everton,1.317380,1.145049,False,1,0.0,0.0,10.0,0.0,0.0,2.0,0.0,0.8,8.4,0.0,0.0,0.0,0.0,1.0,577628.0,1.0,2.0,0.0,-35279.0,2232.0,37511.0,0.0,2020-12-19T17:30:00Z,2021,1.0,NaN,NaN,2020-12-19,Arsenal,Premier League,0.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,29.0,96.6,15.0,0.0,0.0,NaN,NaN,25,41.0,7357,5897.0,44.587879,35.739394,65.0,159.0,21.666667,53.000000,141.0,253.0,28.2,50.6,313.0,496.0,31.3,49.6,766.0,825.0,38.30,41.25,41.0,25.0,6835.0,6627.0,41.424242,40.163636,162.0,79.0,54.000000,26.333333,237.0,176.0,47.4,35.2,391.0,401.0,39.1,40.1,797.0,777.0,39.85,38.850000,6651.00,90.00,180.00,270.00,450.00,900.00,1170.00,265,3.585927,2.0,2.000000,4.0,2.000000,5.0,1.666667,17.0,3.400000,33.0,3.300000,45.0,3.461538
98530,Bernd Leno,15,1,90.00,Arsenal,Chelsea,1.317380,1.859440,True,8,0.0,1.0,27.0,0.0,0.0,1.0,0.0,3.6,35.6,0.0,0.0,1.0,0.0,2.0,532869.0,1.0,3.0,0.0,-44555.0,3358.0,47913.0,0.0,2020-12-26T17:30:00Z,2021,1.0,NaN,NaN,2020-12-26,Arsenal,Premier League,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,31.0,54.8,11.0,0.0,0.0,NaN,NaN,60,23.0,7382,5938.0,44.469880,35.771084,74.0,130.0,24.666667,43.333333,117.0,242.0,23.4,48.4,293.0,510.0,29.3,51.0,717.0,853.0,35.85,42.65,23.0,60.0,8224.0,5698.0,49.542169,34.325301,125.0,125.0,41.666667,41.666667,226.0,206.0,45.2,41.2,53

In [19]:
# create dataset with latest lag numbers for all future gameweeks
valid_season = '2021'
valid_gw = next_gw
valid_len = 38 - next_gw + 1

train_predict_df, train_idx, valid_idx = create_lag_train(lag_train_df, 
                                                      cat_vars, cont_vars, 
                                                      player_lag_vars, team_lag_vars, dep_var,
                                                      valid_season, valid_gw, valid_len)

In [20]:
team_lag_vars

['total_points_team_pg_last_all',
 'total_points_team_pg_last_3',
 'total_points_team_pg_last_5',
 'total_points_team_pg_last_10',
 'total_points_team_pg_last_20',
 'total_points_team_conceded_pg_last_all_opponent',
 'total_points_team_conceded_pg_last_3_opponent',
 'total_points_team_conceded_pg_last_5_opponent',
 'total_points_team_conceded_pg_last_10_opponent',
 'total_points_team_conceded_pg_last_20_opponent']

In [21]:
# take a look at the dataframe
train_predict_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114523 entries, 0 to 114522
Data columns (total 33 columns):
 #   Column                                           Non-Null Count   Dtype   
---  ------                                           --------------   -----   
 0   gw                                               114523 non-null  category
 1   minutes                                          114523 non-null  float64 
 2   minutes_last_1                                   114523 non-null  float64 
 3   minutes_last_10                                  114523 non-null  float64 
 4   minutes_last_2                                   114523 non-null  float64 
 5   minutes_last_20                                  114523 non-null  float64 
 6   minutes_last_3                                   114523 non-null  float64 
 7   minutes_last_5                                   114523 non-null  float64 
 8   minutes_last_all                                 114523 non-null  float64 
 9   oppo

In [22]:
# split out dependent variable
X, y = train_predict_df[cat_vars + cont_vars + player_lag_vars + team_lag_vars].copy(), train_predict_df[dep_var].copy()

In [23]:
# since position is categorical, it should be a string
X['position'] = X['position'].apply(str)

# need to transform season
enc = LabelEncoder()
X['season'] = enc.fit_transform(X['season'])
X_dict = X.to_dict("records")

# Create the DictVectorizer object: dv
dv = DictVectorizer(sparse=False, separator='_')

# Apply dv on df: df_encoded
X_encoded = dv.fit_transform(X_dict)

X_df = pd.DataFrame(X_encoded, columns=dv.feature_names_)

In [24]:
# split out training and prediction sets
X_train = X_df.iloc[train_idx]
y_train = y.iloc[train_idx]
X_test = X_df.iloc[valid_idx]
y_test = y.iloc[valid_idx]

In [25]:
# instatiate and train XGB Regressor
model_params = {"gamma": 0.42,
                "learning_rate": 0.047, 
                "max_depth": 4,
                "n_estimators": 171,
                "subsample": 0.6}

xg_reg = xgb.XGBRegressor(objective="reg:squarederror",
                          gamma=model_params['gamma'], 
                          learning_rate=model_params['learning_rate'], 
                          max_depth=model_params['max_depth'],
                          n_estimators=model_params['n_estimators'],
                          subsample=model_params['subsample'])

xg_reg.fit(X_train, y_train)
preds = xg_reg.predict(X_test)

In [26]:
preds

array([ 6.4584613e-03, -5.0109625e-04,  9.5702469e-01, ...,
        1.7736852e-03,  3.3059134e+00,  1.7736852e-03], dtype=float32)

In [27]:
remaining_season_df['predicted_points'] = preds

In [28]:
remaining_season_df.head(50)

,gw,team,opponent_team,player,position,price,play_proba,short_name,was_home,relative_market_value_team,relative_market_value_opponent_team,season,minutes,total_points,kickoff_time,predicted_points
4731,26,Arsenal,Leicester City,Mesut Özil,3,6.7,0.00,Özil,False,1.252915,1.059550,2021,0.000000,0,9009-99-99,0.006458
4732,26,Arsenal,Leicester City,Sokratis Papastathopoulos,2,4.8,0.00,Sokratis,False,1.252915,1.059550,2021,0.000000,0,9009-99-99,-0.000501
4733,26,Arsenal,Leicester City,David Luiz Moreira Marinho,2,5.4,1.00,David Luiz,False,1.252915,1.059550,2021,58.750000,0,9009-99-99,0.957025
4734,26,Arsenal,Leicester City,Pierre-Emerick Aubameyang,3,11.3,1.00,Aubameyang,False,1.252915,1.059550,2021,70.333333,0,9009-99-99,4.083003
4735,26,Arsenal,Leicester City,Cédric Soares,2,4.6,1.00,Cédric,False,1.252915,1.059550,2021,66.800000,0,9009-99-99,2.576874
4736,26,Arsenal,Leicester City,Alexandre Lacazette,4,8.1,1.00,Lacazette,False,1.252915,1.059550,2021,41.250000,0,9009-99-99,1.771189
4737,26,Arsenal,Leicester City,Shkodran Mustafi,2,5.0,0.00,Mustafi,False,1.252915,1.059550,2021,0.000000,0,9009-99-99,0.002702
4738,26,Arsenal,Leicester City,Bernd Leno,1,5.0,1.00,Leno,False,1.252915,1.059550,2021,85.250000,0,9009-99-99,3.958487
4739,26,Arsenal,Leicester City,Granit Xhaka,3,5.2,1.00,Xhaka,False,1.252915,1.059550,2021,90.000000,0,9009-99-99,2.894504
4740,26,Arsenal,Leicester City,Pablo Marí,2,4.4,1.00,Marí,False,1.252915,1.059550,2021,30.000000,0,9009-99-99,1.088413


In [29]:
remaining_season_df[remaining_season_df['gw'] == next_gw].sort_values(by='predicted_points', ascending=False).head(50)

,gw,team,opponent_team,player,position,price,play_proba,short_name,was_home,relative_market_value_team,relative_market_value_opponent_team,season,minutes,total_points,kickoff_time,predicted_points
4467,26,Tottenham Hotspur,Fulham,Heung-Min Son,3,9.5,1.0,Son,False,1.627545,0.490763,2021,90.000000,0,9001-99-99,7.080659
4465,26,Tottenham Hotspur,Fulham,Harry Kane,4,11.1,1.0,Kane,False,1.627545,0.490763,2021,90.000000,0,9001-99-99,6.957606
209,26,Tottenham Hotspur,Burnley,Heung-Min Son,3,9.5,1.0,Son,True,1.627545,0.342968,2021,90.000000,0,9006-99-99,6.784117
498,26,Manchester City,West Ham United,Raheem Sterling,3,11.6,1.0,Sterling,True,2.397273,0.613679,2021,73.166667,0,9015-99-99,6.693999
4712,26,Liverpool,Sheffield United,Mohamed Salah,3,12.6,1.0,Salah,False,2.532968,0.387635,2021,90.000000,0,9008-99-99,6.620004
207,26,Tottenham Hotspur,Burnley,Harry Kane,4,11.1,1.0,Kane,True,1.627545,0.342968,2021,90.000000,0,9006-99-99,6.614456
464,26,Manchester City,Wolverhampton Wanderers,Raheem Sterling,3,11.6,1.0,Sterling,True,2.397273,0.968408,2021,73.166667,0,9014-99-99,6.248500
4707,26,Liverpool,Sheffield United,Roberto Firmino,4,9.2,1.0,Firmino,False,2.532968,0.387635,2021,73.800000,0,9008-99-99,6.233147
4504,26,Manchester United,Crystal Palace,Bruno Miguel Borges Fernandes,3,11.6,1.0,Fernandes,False,1.669837,0.504106,2021,90.000000,0,9002-99-99,6.176912
147,26,Liverpool,Chelsea,Mohamed Salah,3,12.6,1.0,Salah,True,2.532968,1.817518,2021,90.000000,0,9004-99-99,6.156520


In [30]:
# rename previous week's predictions file
# generate previous week's filename
last_gw = next_gw - 1
filename = 'history/2020-21/predictions_gw' + str(last_gw) + '.csv'
# windows
! move predictions.csv $filename
# linux
# ! mv predictions.csv $filename

        1 file(s) moved.


In [31]:
# write to predictions.csv
## RENAME PREVIOUS WEEK FIRST
predictions = remaining_season_df[['gw', 'player', 'team', 'opponent_team', 
                                   'position', 'price', 'play_proba', 'short_name',
                                   'was_home', 'predicted_points', 'minutes']]
predictions.to_csv('predictions.csv')